<h1 style='color:#008'>Read new BOS csv and insert data on database</h1>

In [14]:
import pandas as pd
import funcs_bos, gc

files_names = funcs_bos.get_files_names(only_not_imported=True)
print('files ok')
files_names

bos = funcs_bos.read_files(files_names)
print('files lidos ok')

bos = funcs_bos.data_bos_processing(bos)
print('processing df ok')

classificadores = funcs_bos.get_classificadores()
bos['SETOR'] = bos.apply(lambda row: funcs_bos.classifica_setor(row, classificadores), axis=1)
print('classificação setores ok')

funcs_bos.classifica_cia(df_bos=bos)
print('classificação cias ok')

bos_db = pd.read_sql_table('tbl_bos', 'sqlite:///gdo.db')
bos_db['TEMPO_DT']  = bos_db['DTA_HRA_TERMINO_DT'] - bos_db['DTA_HRA_INICIO_DT']
print('bos_db criado')

bos_total = pd.concat([bos_db, bos])
bos_total = bos_total.drop_duplicates(subset='RAT.NUM_ATIVIDADE', keep='last')

bos_total.to_sql('tbl_bos', 'sqlite:///gdo.db', if_exists='replace', index=False)
print('dados exportados para o db ok')

pd.DataFrame(files_names).to_sql('tbl_imported_files_bos', 'sqlite:///gdo.db', if_exists='append', index=False)
print('nomes dos arquivos importados, já inseridos no db')

del bos, bos_total
print('bos deleted')
gc.collect()

files ok


Exception: Não há arquivos csv novos para serem inseridos no banco de dados.